<a href="https://colab.research.google.com/github/AbhiSrvstv/Q-A-Chatbot/blob/main/Loan%20Predcition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio

In [2]:
import gradio as gr
import requests
import json
import time
from google.colab import userdata # Import userdata to access Colab secrets

# Function to call the Gemini API for loan risk prediction
def predict_loan_risk(
    applicant_age: int,
    applicant_income: float,
    loan_amount: float,
    loan_term: int,
    credit_score: int,
    employment_length: int,
    home_ownership: str,
    loan_purpose: str,
    existing_monthly_debts: float,
    risk_agent_type: str # New parameter for multi-agent functionality
) -> str:
    """
    Predicts loan risk using the Gemini API based on provided loan application details.

    Args:
        applicant_age (int): Age of the applicant in years.
        applicant_income (float): Annual income of the applicant.
        loan_amount (float): Requested loan amount.
        loan_term (int): Loan term in months.
        credit_score (int): Credit score of the applicant (e.g., FICO).
        employment_length (int): Employment length in years.
        home_ownership (str): Type of home ownership (Rent, Own, Mortgage, Other).
        loan_purpose (str): Purpose of the loan (Debt Consolidation, Education, Home Improvement, Venture, Other).
        existing_monthly_debts (float): Total existing monthly debt payments.
        risk_agent_type (str): The type of risk assessment agent to use (e.g., 'General', 'Financial Stability', 'Credit History').

    Returns:
        str: A formatted string containing the predicted loan risk and justification.
    """
    # Retrieve the API key from Colab secrets using userdata
    # Your secret name is 'GOOGLE_API_KEY'
    api_key = userdata.get("GOOGLE_API_KEY")

    if not api_key:
        return "Error: Gemini API key not found. Please ensure your secret 'GOOGLE_API_KEY' is set up in Colab secrets and notebook access is enabled."

    # Base prompt for the Gemini model
    base_prompt = f"""
    Analyze the following loan application details and predict the loan risk as "Low Risk", "Medium Risk", or "High Risk".
    Provide a detailed justification for your prediction, explaining the key factors that led to the assessment.

    Loan Application Details:
    - Applicant Age: {applicant_age} years
    - Applicant Income: ${applicant_income} per year
    - Loan Amount: ${loan_amount}
    - Loan Term: {loan_term} months
    - Credit Score: {credit_score} (out of 850)
    - Employment Length: {employment_length} years
    - Home Ownership: {home_ownership}
    - Loan Purpose: {loan_purpose}
    - Existing Monthly Debts: ${existing_monthly_debts}
    """

    # Customize prompt based on agent type
    if risk_agent_type == "Financial Stability Agent":
        agent_specific_instruction = "Focus your justification primarily on the applicant's income, existing debts, and the loan amount relative to their financial capacity."
    elif risk_agent_type == "Credit History Agent":
        agent_specific_instruction = "Focus your justification primarily on the applicant's credit score and employment stability."
    else: # Default to General Risk Agent
        agent_specific_instruction = "Provide a comprehensive justification considering all relevant factors."

    prompt = f"""
    {base_prompt}
    {agent_specific_instruction}

    Please format your response strictly as follows:
    Risk: [Your Prediction]
    Justification: [Your Detailed Justification]
    """

    chat_history = [{"role": "user", "parts": [{"text": prompt}]}]

    payload = {
        "contents": chat_history,
        "generationConfig": {
            "responseMimeType": "text/plain",
            "temperature": 0.7, # Added temperature to encourage more varied responses
        },
    }

    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent?key={api_key}"

    retries = 0
    max_retries = 5
    base_delay = 1  # 1 second

    while retries < max_retries:
        try:
            response = requests.post(
                api_url,
                headers={'Content-Type': 'application/json'},
                data=json.dumps(payload)
            )
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

            result = response.json()

            # Check for content filtering or empty candidates
            if result.get("candidates") and len(result["candidates"]) > 0:
                if result["candidates"][0].get("finishReason") == "SAFETY":
                    return "Prediction blocked due to safety concerns. Please adjust input or prompt."
                if result["candidates"][0].get("content") and \
                   result["candidates"][0]["content"].get("parts") and \
                   len(result["candidates"][0]["content"]["parts"]) > 0:
                    text = result["candidates"][0]["content"]["parts"][0]["text"]

                    # Robust parsing for Risk and Justification
                    prediction_text = "N/A"
                    justification_text = "The model did not provide a specific justification for this prediction."

                    # Find the start of "Risk:" and "Justification:"
                    risk_start_idx = text.find("Risk:")
                    justification_start_idx = text.find("Justification:")

                    if risk_start_idx != -1:
                        # Extract prediction text between "Risk:" and "Justification:" or end of string
                        if justification_start_idx != -1 and justification_start_idx > risk_start_idx:
                            prediction_text = text[risk_start_idx + len("Risk:"):justification_start_idx].strip()
                        else:
                            prediction_text = text[risk_start_idx + len("Risk:"):].strip()
                        # Clean up prediction_text if it contains "Justification:"
                        if "Justification:" in prediction_text:
                            prediction_text = prediction_text.split("Justification:")[0].strip()


                    if justification_start_idx != -1:
                        # Extract everything after "Justification:"
                        justification_text = text[justification_start_idx + len("Justification:"):].strip()
                        if not justification_text: # If it's still empty, use default
                            justification_text = "The model did not provide a specific justification for this prediction."


                    return (
                        f"**Predicted Risk ({risk_agent_type}):** {prediction_text}\n\n"
                        f"**Justification:** {justification_text}"
                    )
                else:
                    return "Error: Gemini API returned an empty content part. This might indicate an issue with the model's generation."
            else:
                return "Error: Gemini API returned no candidates for prediction. This could be due to content filtering or an internal model error."

        except requests.exceptions.HTTPError as e:
            status_code = e.response.status_code
            error_detail = e.response.text
            if status_code == 400:
                return f"API Error (400 Bad Request): Invalid input or request format. Details: {error_detail}"
            elif status_code == 403:
                return f"API Error (403 Forbidden): Authentication failed. Check your API key ('GOOGLE_API_KEY') or ensure it's enabled for the Gemini API. Details: {error_detail}"
            elif status_code == 429:
                return f"API Error (429 Too Many Requests): Rate limit exceeded. Please wait and try again. Details: {error_detail}"
            elif status_code == 500:
                return f"API Error (500 Internal Server Error): Gemini API experienced an internal error. Details: {error_detail}"
            else:
                return f"API HTTP Error ({status_code}): {e.response.reason}. Details: {error_detail}"
        except requests.exceptions.ConnectionError as e:
            return f"Network Error: Could not connect to the Gemini API. Check your internet connection or the API endpoint. Details: {e}"
        except requests.exceptions.Timeout as e:
            return f"Network Error: The request to the Gemini API timed out. Details: {e}"
        except requests.exceptions.RequestException as e:
            # Catch all other requests exceptions
            return f"API Request Error: An unexpected issue occurred during the API call. Details: {e}"
        except json.JSONDecodeError:
            return "Error: Could not decode JSON response from API. The API might have returned an invalid or malformed response."
        except Exception as e:
            # Catch any other unexpected Python errors
            return f"An unexpected error occurred during prediction: {e}"

    return "Error: Failed to predict loan risk after multiple retries. Please try again later."


# Define Gradio Interface
iface = gr.Interface(
    fn=predict_loan_risk,
    inputs=[
        gr.Number(label="Applicant Age (years)", minimum=18, value=30),
        gr.Number(label="Annual Applicant Income ($)", minimum=0, value=60000),
        gr.Number(label="Loan Amount ($)", minimum=1, value=15000),
        gr.Number(label="Loan Term (months)", minimum=1, value=36),
        gr.Number(label="Credit Score (300-850)", minimum=300, maximum=850, value=720),
        gr.Number(label="Employment Length (years)", minimum=0, value=5),
        gr.Dropdown(
            # Expanded Home Ownership options
            ["Rent", "Own", "Mortgage", "Living with Parents", "Other"],
            label="Home Ownership",
            value="Rent"
        ),
        gr.Dropdown(
            # Expanded Loan Purpose options
            ["Debt Consolidation", "Education", "Home Improvement", "Venture",
             "Auto Loan", "Medical Expenses", "Vacation", "Business Expansion", "Other"],
            label="Loan Purpose",
            value="Debt Consolidation"
        ),
        gr.Number(label="Existing Monthly Debts ($)", minimum=0, value=500),
        gr.Dropdown( # New dropdown for agent selection
            ["General Risk Agent", "Financial Stability Agent", "Credit History Agent"],
            label="Risk Assessment Agent",
            value="General Risk Agent",
            info="Select an agent to get a specific perspective on loan risk."
        )
    ],
    outputs=gr.Markdown(label="Loan Risk Prediction"),
    title="Gen AI Loan Risk Predictor",
    description="Enter the loan application details to get a risk assessment using Generative AI (Gemini).",
    css="""
    body { font-family: 'Inter', sans-serif; }
    .gradio-container { border-radius: 12px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); }
    h1 { color: #1f2937; font-weight: 800; }
    p { color: #4b5563; }
    button { background-color: #3b82f6 !important; color: white !important; border-radius: 0.375rem !important; }
    button:hover { background-color: #2563eb !important; }
    input[type="number"], select { border-radius: 0.375rem; border: 1px solid #d1d5db; padding: 0.5rem 0.75rem; }
    input[type="number"]:focus, select:focus { outline: none; border-color: #3b82f6; ring: 2px; ring-color: #93c5fd; }
    .gr-box { border-radius: 0.5rem; }
    .gr-button { border-radius: 0.5rem; }
    """
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch(share=True) # Set share=True to get a public link (useful for testing)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a5c666b605927a5de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import gradio as gr
import requests
import json
import time
from google.colab import userdata  # Only works in Colab

# Function to call the Gemini API
def predict_loan_risk(
    applicant_age: int,
    applicant_income: float,
    loan_amount: float,
    loan_term: int,
    credit_score: int,
    employment_length: int,
    home_ownership: str,
    loan_purpose: str,
    existing_monthly_debts: float,
    risk_agent_type: str
) -> str:
    api_key = userdata.get("GOOGLE_API_KEY")

    if not api_key:
        return "Error: Gemini API key not found. Please ensure your secret 'GOOGLE_API_KEY' is set up in Colab secrets and notebook access is enabled."

    base_prompt = f"""
    Analyze the following loan application details and predict the loan risk as "Low Risk", "Medium Risk", or "High Risk".
    Provide a detailed justification for your prediction, explaining the key factors that led to the assessment.

    Loan Application Details:
    - Applicant Age: {applicant_age} years
    - Applicant Income: ${applicant_income} per year
    - Loan Amount: ${loan_amount}
    - Loan Term: {loan_term} months
    - Credit Score: {credit_score} (out of 850)
    - Employment Length: {employment_length} years
    - Home Ownership: {home_ownership}
    - Loan Purpose: {loan_purpose}
    - Existing Monthly Debts: ${existing_monthly_debts}
    """

    if risk_agent_type == "Financial Stability Agent":
        agent_specific_instruction = "Focus your justification primarily on the applicant's income, existing debts, and the loan amount relative to their financial capacity."
    elif risk_agent_type == "Credit History Agent":
        agent_specific_instruction = "Focus your justification primarily on the applicant's credit score and employment stability."
    else:
        agent_specific_instruction = "Provide a comprehensive justification considering all relevant factors."

    prompt = f"""
    {base_prompt}
    {agent_specific_instruction}

    Please format your response strictly as follows:
    Risk: [Your Prediction]
    Justification: [Your Detailed Justification]
    """

    chat_history = [{"role": "user", "parts": [{"text": prompt}]}]

    payload = {
        "contents": chat_history,
        "generationConfig": {
            "responseMimeType": "text/plain",
            "temperature": 0.7,
        },
    }

    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent?key={api_key}"

    retries = 0
    max_retries = 5
    base_delay = 1

    while retries < max_retries:
        try:
            response = requests.post(
                api_url,
                headers={'Content-Type': 'application/json'},
                data=json.dumps(payload)
            )
            response.raise_for_status()
            result = response.json()

            if result.get("candidates") and len(result["candidates"]) > 0:
                if result["candidates"][0].get("finishReason") == "SAFETY":
                    return "Prediction blocked due to safety concerns."

                if result["candidates"][0].get("content") and \
                   result["candidates"][0]["content"].get("parts") and \
                   len(result["candidates"][0]["content"]["parts"]) > 0:

                    text = result["candidates"][0]["content"]["parts"][0]["text"]
                    prediction_text = "N/A"
                    justification_text = "The model did not provide a specific justification for this prediction."

                    risk_start_idx = text.find("Risk:")
                    justification_start_idx = text.find("Justification:")

                    if risk_start_idx != -1:
                        if justification_start_idx != -1 and justification_start_idx > risk_start_idx:
                            prediction_text = text[risk_start_idx + len("Risk:"):justification_start_idx].strip()
                        else:
                            prediction_text = text[risk_start_idx + len("Risk:"):].strip()
                        if "Justification:" in prediction_text:
                            prediction_text = prediction_text.split("Justification:")[0].strip()

                    if justification_start_idx != -1:
                        justification_text = text[justification_start_idx + len("Justification:"):].strip()
                        if not justification_text:
                            justification_text = "The model did not provide a specific justification for this prediction."

                    return (
                        f"**Predicted Risk ({risk_agent_type}):** {prediction_text}\n\n"
                        f"**Justification:** {justification_text}"
                    )
                else:
                    return "Error: Gemini API returned an empty content part."
            else:
                return "Error: Gemini API returned no candidates."

        except requests.exceptions.RequestException as e:
            return f"API Request Error: {e}"
        except json.JSONDecodeError:
            return "Error: Could not decode JSON response."
        except Exception as e:
            return f"Unexpected error: {e}"

    return "Error: Failed to predict loan risk after multiple retries."

# Gradio Interface
iface = gr.Interface(
    fn=predict_loan_risk,
    inputs=[
        gr.Number(label="Applicant Age (years)", minimum=18, value=30),
        gr.Number(label="Annual Applicant Income ($)", minimum=0, value=60000),
        gr.Number(label="Loan Amount ($)", minimum=1, value=15000),
        gr.Number(label="Loan Term (months)", minimum=1, value=36),
        gr.Number(label="Credit Score (300-850)", minimum=300, maximum=850, value=720),
        gr.Number(label="Employment Length (years)", minimum=0, value=5),
        gr.Dropdown(
            ["Rent", "Own", "Mortgage", "Living with Parents", "Other"],
            label="Home Ownership",
            value="Rent"
        ),
        gr.Dropdown(
            ["Debt Consolidation", "Education", "Home Improvement", "Venture",
             "Auto Loan", "Medical Expenses", "Vacation", "Business Expansion", "Other"],
            label="Loan Purpose",
            value="Debt Consolidation"
        ),
        gr.Number(label="Existing Monthly Debts ($)", minimum=0, value=500),
        gr.Dropdown(
            ["General Risk Agent", "Financial Stability Agent", "Credit History Agent"],
            label="Risk Assessment Agent",
            value="General Risk Agent",
            info="Select an agent to get a specific perspective on loan risk."
        )
    ],
    outputs=gr.Markdown(label="Loan Risk Prediction"),
    title="Gen AI Loan Risk Predictor",
    description="Enter the loan application details to get a risk assessment using Generative AI (Gemini).",
    flagging_dir="flagged_data",  # 🔥 Enables CSV flag logging
    allow_flagging="manual",      # "manual" enables user-initiated flagging
    css="""
    body { font-family: 'Inter', sans-serif; }
    .gradio-container { border-radius: 12px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); }
    h1 { color: #1f2937; font-weight: 800; }
    p { color: #4b5563; }
    button { background-color: #3b82f6 !important; color: white !important; border-radius: 0.375rem !important; }
    button:hover { background-color: #2563eb !important; }
    input[type="number"], select { border-radius: 0.375rem; border: 1px solid #d1d5db; padding: 0.5rem 0.75rem; }
    input[type="number"]:focus, select:focus { outline: none; border-color: #3b82f6; ring: 2px; ring-color: #93c5fd; }
    .gr-box { border-radius: 0.5rem; }
    .gr-button { border-radius: 0.5rem; }
    """
)

# Launch app
if __name__ == "__main__":
    iface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a0b131931951f28f08.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
